<h1>COLLABORATION BASED RECOMMENDATION SYSTEM

<h3>Importing libraries

In [367]:
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import random

<h3>Task 1.Importing Dataset

In [368]:
movies_data=pd.read_csv('movies_metadata.csv',low_memory=False)
links=pd.read_csv('links_small.csv',low_memory=False)
ratings=pd.read_csv('ratings_small.csv',low_memory=False)

<h3>TAsk2. Exploring Datasets

In [369]:
#this is how movies_metadata lookslike
movies_data.head(2)

adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   

                               homepage    id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story   862  tt0114709                en   
1                                   NaN  8844  tt0113497                en   

  original_title                                           overview  \
0      Toy Story  Led by Woody, Andy's toys live happily in his ...   
1        Jumanji  When siblings Judy and Peter discover an encha...   

     ...     release_date      revenue runtime  \
0    ...       1995-10-30  373554033.0    81.0   
1    ...       1995-12-15  262797249.0   104.0   

                                    spoken_languages    status  \
0           [{'iso_639_1': 'en', 'name': 'English'}]  Released   
1  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...  Released   

                                     tagline      title  video vote_average  \
0                                        NaN  Toy Story  False          7.7   
1  Roll the dice and unleash the excitement!    Jumanji  False          6.9   

  vote_count  
0     5415.0  
1     2413.0  

[2 rows x 24 columns]

In [370]:
#basic statistical analysis
movies.describe()

id       revenue       runtime  vote_average    vote_count
count   45463.000000  4.546000e+04  45203.000000  45460.000000  45460.000000
mean   108359.918813  1.120935e+07     94.128199      5.618207    109.897338
std    112460.749278  6.433225e+07     38.407810      1.924216    491.310374
min         2.000000  0.000000e+00      0.000000      0.000000      0.000000
25%     26449.500000  0.000000e+00     85.000000      5.000000      3.000000
50%     60003.000000  0.000000e+00     95.000000      6.000000     10.000000
75%    157328.000000  0.000000e+00    107.000000      6.800000     34.000000
max    469172.000000  2.787965e+09   1256.000000     10.000000  14075.000000

In [371]:
#columns with their corresponding null values
movies.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   3
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      3
runtime                    260
spoken_languages             3
status                      84
tagline                  25051
title                        3
video                        3
vote_average                 3
vote_count                   3
dtype: int64

In [372]:
#this is how links_small lookslike
links.head(2)

movieId  imdbId  tmdbId
0        1  114709   862.0
1        2  113497  8844.0

In [373]:
#basic statistical analysis
links.describe()

movieId        imdbId         tmdbId
count    9125.000000  9.125000e+03    9112.000000
mean    31123.291836  4.798244e+05   39104.545544
std     40782.633604  7.431774e+05   62814.519801
min         1.000000  4.170000e+02       2.000000
25%      2850.000000  8.884600e+04    9451.750000
50%      6290.000000  1.197780e+05   15852.000000
75%     56274.000000  4.284410e+05   39160.500000
max    164979.000000  5.794766e+06  416437.000000

In [374]:
#columns with their corresponding null values
links.isna().sum()

movieId     0
imdbId      0
tmdbId     13
dtype: int64

In [375]:
#this is how ratings_small lookslike
ratings.head()

userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205

In [376]:
#basic statistical analysis
ratings.describe()

userId        movieId         rating     timestamp
count  100004.000000  100004.000000  100004.000000  1.000040e+05
mean      347.011310   12548.664363       3.543608  1.129639e+09
std       195.163838   26369.198969       1.058064  1.916858e+08
min         1.000000       1.000000       0.500000  7.896520e+08
25%       182.000000    1028.000000       3.000000  9.658478e+08
50%       367.000000    2406.500000       4.000000  1.110422e+09
75%       520.000000    5418.000000       4.000000  1.296192e+09
max       671.000000  163949.000000       5.000000  1.476641e+09

In [377]:
#columns with their corresponding null values
ratings.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

<h3>Task 3. Data cleaning

In [378]:
#mergining the dataset of ratings with links on the basis of movieId
cleaned_data=links.merge(ratings,left_on='movieId', right_on='movieId')

In [379]:
#dataframe after merging
cleaned_data.head()

movieId  imdbId  tmdbId  userId  rating   timestamp
0        1  114709   862.0       7     3.0   851866703
1        1  114709   862.0       9     4.0   938629179
2        1  114709   862.0      13     5.0  1331380058
3        1  114709   862.0      15     2.0   997938310
4        1  114709   862.0      19     3.0   855190091

In [380]:
#checking the id column of movies dataframe to be of type numeric or not
is_numeric_dtype(movies_data['id'])

False

In [381]:
#removing the string values in the id column
movies_data=movies_data.drop(19730)#19730
movies_data=movies_data.drop(29503)#29503
movies_data=movies_data.drop(35587)#35587
#converting the id column into type int
movies_data['id']=movies_data['id'].astype(int)

In [382]:
#checking numeric type after converting it into type int
is_numeric_dtype(movies_data['id'])

True

In [383]:
#checking for any null values
movies_data['id'].isna().sum()

0

<h3>Task 4. Data Wrangling

In [384]:
#mergining the dataset of cleaned_data with movies_data on the basis of movieId and id 
cleaned_data=cleaned_data.merge(movies_data,left_on='movieId', right_on='id')

In [385]:
#checking for null values in tmdbid 
cleaned_data['tmdbId'].isna().sum()

14

In [386]:
#filling the null values with 0
cleaned_data['tmdbId']=cleaned_data['tmdbId'].fillna(0)
#converting the typr to int
cleaned_data['tmdbId']=cleaned_data['tmdbId'].astype(int)

In [387]:

cleaned_data['tmdbId'].isna().sum()

0

In [388]:
#using the pivot_table function to convert the table in such a way that index=title,columns=userId and values=ratings of users 
wrangled_data=cleaned_data.pivot_table(index='title',columns='userId',values='rating').fillna(0)

<h3>Task 5. Preparing data for Collaboration Based Recommender

In [389]:
#checking after converting
wrangled_data.tail()

userId            1    2    3    4    5    6    7    8    9    10  ...   662  \
title                                                              ...         
xXx               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0   
¡Three Amigos!    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0   
À nos amours      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0   
Ödipussi          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0   
Şaban Oğlu Şaban  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0   

userId            663  664  665  666  667  668  669  670  671  
title                                                          
xXx               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
¡Three Amigos!    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
À nos amours      0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  
Ödipussi          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
Şaban Oğlu Şaban  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 671 columns]

In [390]:
#converting the dataframe values to csr_matrix to save space
mat_movie_features=csr_matrix(wrangled_data.values)


In [391]:
#creating the object of knn algorithm
knn=NearestNeighbors(metric='cosine',algorithm='brute')

In [392]:
knn

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

<h3>Task 6. Building a Collaboration Based recommendation system

In [393]:
#function toshow recommendation by taking random movie
def collaboration_recommender():
    query_index=np.random.choice(2794)
    knn.fit(mat_movie_features,)
    distances,indices=knn.kneighbors(wrangled_data.iloc[query_index,:].values.reshape(1, -1),n_neighbors=20)
    df=pd.DataFrame()
    for i in indices.flatten():
        df=df.append(wrangled_data.iloc[i])
    print("for the movie:",' "',wrangled_data.iloc[query_index,:].name,'"',"recommendations are:",'\n')
    for i in df.index:
        if i==wrangled_data.iloc[query_index,:].name:
            continue
        else:
            print(i)
    

In [394]:
collaboration_recommender()

for the movie:  " Entrapment " recommendations are: 

This Island Earth
Insomnia
Murder Most Foul
Abel
In Bed
Tomboy
The Cider House Rules
Blondie Goes Latin
The Wizard
Blondie Plays Cupid
Crash
Constantine
Emma's Bliss
The Real McCoy
In Good Company
What Lies Beneath
Paranoid Park
Sin City: A Dame to Kill For
Games of Love and Chance


In [395]:
#function toshow recommendation by inputting name of the  movie as argument
def collaboration_recommender1(name):
    knn.fit(mat_movie_features,)
    distances,indices=knn.kneighbors(wrangled_data.loc[name,:].values.reshape(1, -1),n_neighbors=20)
    df=pd.DataFrame()
    for i in indices.flatten():
        df=df.append(wrangled_data.iloc[i])
    print("for the movie:",' "',wrangled_data.loc[name,:].name,'"',"recommendations are:",'\n')
    for i in df.index:
        if i==wrangled_data.loc[name,:].name:
            continue
        else:
            print(i)
    

In [396]:
collaboration_recommender1('xXx')

for the movie:  " xXx " recommendations are: 

Cheerleaders' Wild Weekend
The Night Child
The Mystery of Chess Boxing
Curtains
New York Doll
Five Times Two
Horrible Bosses
Big Girls Don't Cry... They Get Even
Affair in Havana
Waiter
Ghost Dog: The Way of the Samurai
The Mikado
Manitou's Shoe
Godzilla vs. Mechagodzilla II
Savior
The Dawn Patrol
My Name Is Joe
Fools Rush In
Cowboy
